## Introdução

Você já ouviu falar do Conjunto de Mandelbrot1? Seu descobridor foi Benoit Mandelbrot, que trabalhava na IBM durante a década de 1960 e foi um dos primeiros a usar computação gráfica para mostrar como complexidade pode surgir a partir de regras simples. Benoit fez isso gerando e visualizando imagens de geometria fractal.
Um desses fractais foi nomeado Conjunto de Mandelbrot pelo matemático Adrien Douady. O Conjunto de Mandelbrot pode ser informalmente definido como o conjunto dos números complexos c para os quais a função fc(z) = z2 + x não diverge quando é iterada começando em z = 0. Isto é, a sequência fc(0), fc(fc(0)), fc(fc(fc(0))), . . . é sempre limitada. A Figura 1 mostra uma região do Conjunto de Mandelbrot conhecida como Seahorse Valley.

![image info](./doc/img/elephant.png)


## Metodologia

- falar da paralelização do programa em Ptreads e OpenMP e particularidades
- falar sobre os experimentos, onde foi adaptado o programa run measuremente para aceitar o numero de threads e feito 10 medicoes e captura o tempo médio e desvio padrão com auxílio do 'perf stat'
- Vocês devem analisar também o impacto das porções não paralelizáveis do código sequencial: as operações de I/O e alocação de memória. Uma vez que você verifique que as versões parale- lizadas produzem o resultado correto, elas não precisam realizar I/O e alocação de memória nos testes de desempenho, pois esses custos são fixos e assim aceleramos os experimentos.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-0pky">Pthreads OpenMP Sequencial</th>
    <th class="tg-0pky"></th>
    <th class="tg-0pky"></th>
    <th class="tg-0pky"></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Regiões</td>
    <td class="tg-0pky">Triple Spiral, Elephant, Seahorse &amp; Full</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">I/O e Aloc. Mem.</td>
    <td class="tg-0pky">Sem</td>
    <td class="tg-0pky">Com e Sem</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">No de Threads</td>
    <td class="tg-0pky">2^2 ... 2^5</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">Tamanho da Entrada</td>
    <td class="tg-0pky">2^4 ... 2^{13}</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">No de Execuções</td>
    <td class="tg-0pky">10</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
</tbody>
</table>

## Resultados e Análises

In [2]:
using Pkg
Pkg.add("Plots")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [3]:
Pkg.add("PyPlot")
using Plots

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [4]:
Pkg.add("CSV")
using CSV

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [5]:
Pkg.add("DataFrames")
using DataFrames

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


### Sequencial



### Pthreads

Conforme mostram os resultados obtidos por nossos experimentos nos gráficos abaixo, o ganho de performance do paralelismo em relação ao desempenho sequencial (aumento do número de threads) começa a ser percebido a partir do tamanho de entrada N = 2^9; aonde, ao gerar uma visualização das regiões mais custosas computacionalmente (seahorse, elephant e triple_spiral), vemos uma redução de 0.1 segundo para 2^1 threads utilizadas, até 0.2 segundos para o máximo de threads explicitado no enunciado. 

Em relação à imagem completa do conjunto (full), devido a rapidez de sua geração mesmo na versão sequencial do programa, algo em torno de 10 segundos para N = 2^13, vimos como mais pertinente, para efeitos de comparação, focar a análise apenas nas outras 3 regiões de Mandelbrot, que demoram cerca de 1 minuto para serem criadas na versão sequencial do código fornecido. 

É importante ressaltar que os ganhos de performance gerados pelo paralelismo não foram lineares. Essa característica é claramente demonstrada ao compararmos a performance do programa sequencial com as entradas 2^12 e 2^13 (por fins de clareza, as chamemos de N1 e N2, respectivamente): Analisando os logs da região triple_spiral, enquanto N1 consegue uma melhora de desempenho de 42% (˜6 segundos) ao dividirmos as tarefas de 1 para 2 threads, o programa executa cerca de 60% mais rápido (˜5 segundos) com de 2 para 8 threads, mas melhora apenas 20% (~0.7 segundos) entre 8 e 32 threads.  

In [6]:
inputlabel = [16,32,64,128,256,512,1024,2048,4096,8192]
plotly()
ts = CSV.read("src/results/mandelbrot_pth/triple_spiral.csv", DataFrame)
sh = CSV.read("src/results/mandelbrot_pth/seahorse.csv", DataFrame)
full = CSV.read("src/results/mandelbrot_pth/full.csv", DataFrame)
elephant = CSV.read("src/results/mandelbrot_pth/elephant.csv", DataFrame)
pts = plot(ts.sizes, ts.avg_time, group=ts.threads, xlabel = inputlabel,linewidth = 4,title="Desempenho - triple_spiral",legendtitle="Número de threads")
psh = plot(sh.sizes, sh.avg_time, group=sh.threads, xlabel = inputlabel,linewidth = 4,title="Desempenho - seahorse",legendtitle="Número de threads")
pfull = plot(full.sizes, full.avg_time, group=full.threads, xlabel = inputlabel,linewidth = 4,title="Desempenho - full",legendtitle="Número de threads")
pel = plot(elephant.sizes, elephant.avg_time, group=elephant.threads, xlabel = inputlabel,linewidth = 4,title="Desempenho - elephant",legendtitle="Número de threads")
plot(pts,psh,pfull,pel,layout = @layout([pts psh; pfull pel]),size=(1200,1200))
xlabel!("Tamanho de entrada")
ylabel!("Tempo em segundos")


┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/gabrielaraujo/.julia/packages/Plots/E3MWZ/src/backends.jl:373


<!DOCTYPE html>
 
 
 Plots.jl

### OpenMP